# Programming Assignment 2

**Name:** <br />
**Roll No:** 
***

## Instructions

- **Release Date**: **10th May 2024**  
- **Deadline**: **22nd May 2024 11:59PM**
- Kindly name your submission files as `RollNo_Name_PA2.ipynb`.  <br />
- You are required to work out your answers and submit only the iPython Notebook. The code should be well commented and easy to understand as there are marks for this. This notebook can be used as a template for assignment submission. <br />
- Submissions are to be made through iPearl portal. Submissions made through mail will not be graded.<br />
- Answers to the theory questions if any should be included in the notebook itself. While using special symbols use the $\LaTeX$ mode <br />
- Make sure your plots are clear and have title, legends and clear lines, etc. <br />
- Plagiarism of any form will not be tolerated. If your solutions are found to match with other students or from other uncited sources, there will be heavy penalties and the incident will be reported to the disciplinary authorities. <br />
- In case you have any doubts, feel free to reach out to TAs for help. <br />

***

## E1: A Deterministic Career Path

Consider a simple Markov Decision Process below with four states and two actions available at each state. In this simplistic setting actions have deterministic effects, i.e., taking an action in a state always leads to one next state with transition probability equal to one. There are two actions out of each state for the agent to choose from: D for development and R for research. The _ultimately-care-only-about-money_ reward scheme is given along with the states.

<img src='assets/mdp-d.png' width="700" align="left"></img>

In [1]:
# import required libraries
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random

### E1.1 Environment Implementation

In [2]:
'''
Represents a Career Path problem Gym Environment which provides a Fully observable
MDP
'''
class CareerPathEnv(gym.Env):
    '''
    CareerPathEnv represents the Gym Environment for the Career Path problem environment
    States : [0:'Unemployed',1:'Industry',2:'Grad School',3:'Academia']
    Actions : [0:'Research', 1:'Development']
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=0,no_states=4,no_actions=2):
        '''
        Constructor for the CareerPath class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 4
            no_actions : The no. of possible actions which is 2
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.prob_dynamics = {
            # s: {
            #   a: [(p(s,s'|a), s', r', terminal/not)]    
            # }

            0: {
                0: [(1.0, 2, 0.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            1: {
                0: [(1.0, 0, -10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            2: {
                0: [(1.0, 3, 10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            3: {
                0: [(1.0, 3, 10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(1.0, 2, 0.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            1: {
                0: [(1.0, 0, -10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            2: {
                0: [(1.0, 3, 10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            3: {
                0: [(1.0, 3, 10.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        dynamics_tuple = self.prob_dynamics[self.state][action][0]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

### E1.2 Policies

After implementing the environment let us see how to make decisions in the environment. Let $\pi_1(s) = R$ and $\pi_2(s) = D$ for any state be two policies. Let us see how these policies look like.

In [3]:
policy_R = np.concatenate((np.ones([4, 1]), np.zeros([4, 1])), axis=1)
policy_D = np.concatenate((np.zeros([4, 1]), np.ones([4, 1])), axis=1)
policy_random = np.array((np.random.permutation(2), np.random.permutation(2), np.random.permutation(2), np.random.permutation(2)))
print("Research policy: \n",policy_R)
print("Development policy: \n", policy_D)
print("Random policy: \n",policy_random)

policy_uncertain = np.concatenate((0.5*np.ones([4, 1]), 0.5*np.ones([4, 1])), axis=1)
print("Uncertain policy: \n",policy_uncertain)

Research policy: 
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Development policy: 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
Random policy: 
 [[0 1]
 [1 0]
 [0 1]
 [0 1]]
Uncertain policy: 
 [[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


### E1.3 Testing

By usine one of the above policies, lets see how we navigate the environment. We want to see how we make take and action based on a given policy, what state we transition to and obtain the rewards from the transition.

In [4]:
env = CareerPathEnv()
is_Terminal = False
start_state = env.reset()
steps = 0
total_reward = 0

# you may change policy here
policy = policy_uncertain
# policy = policy_R
# policy = policy_D
# policy = policy_random

print("State\t", "Action\t" , "New State\t" , "Reward\t" , "is_Terminal")
steps = 0
max_steps = 5

prev_state = start_state

while steps < 10:
    steps += 1

    action = np.random.choice(2,1,p=policy[prev_state])[0]  #0 -> Research, 1 -> Development
    state, reward, is_Terminal = env.step(action)
    
    total_reward += reward

    print(" ",prev_state, "\t  ", action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    prev_state = state
    
print("Total Number of steps:", steps)
print("Final Reward:", total_reward)

State	 Action	 New State	 Reward	 is_Terminal
  0 	   1 	   1 	 100.0 	   False
  1 	   0 	   0 	 -10.0 	   False
  0 	   0 	   2 	 0.0 	   False
  2 	   0 	   3 	 10.0 	   False
  3 	   0 	   3 	 10.0 	   False
  3 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   0 	   0 	 -10.0 	   False
  0 	   0 	   2 	 0.0 	   False
  2 	   0 	   3 	 10.0 	   False
Total Number of steps: 10
Final Reward: 310.0


### Iterative Policy Evaluation
Iterative Policy Evaluation is commonly used to calculate the state value function $V_\pi(s)$ for a given policy $\pi$. Here we implement a function to compute the state value function $V_\pi(s)$ for a given policy

<img src='assets/policy_eval.png' width="500" align="left"></img>

In [5]:
# Policy Evaluation
def EvaluatePolicy(env, policy, gamma=0.9, theta=1e-8, draw=False):
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            Vs = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.P()[s][a]:
                    Vs += action_prob * prob * (reward + gamma * V[next_state])
            delta = max(delta, np.abs(V[s]-Vs))
            V[s] = Vs
        if delta < theta:
            break
    return V

### Policy improvement

$\pi'(s) = \arg \max_a \sum_{s',r} p(s',r|s,a)\left[ r + \gamma v_\pi(s') \right ]$


In [6]:
##Policy Improvement Function
def ImprovePolicy(env, v, gamma):
    num_states = env.nS
    num_actions = env.nA
    prob_dynamics = env.P()

    q = np.zeros((num_states, num_actions))
    
    for state in prob_dynamics:
            for action in prob_dynamics[state]:
                #print(state, action)
                for prob, new_state, reward, is_terminal in prob_dynamics[state][action]:
                    #print(prob, new_state, reward, is_terminal)
                    q[state][action] += prob*(reward + gamma*v[new_state])
                        
    new_pi = np.zeros((num_states, num_actions))
    
    for state in range(num_states):
        opt_action = np.argmax(q[state])
        new_pi[state][opt_action] = 1.0
    
    return new_pi       

### Policy Iteration

<img src='assets/policy_iteration.png' width="500" align="left"></img>

In [7]:
def PolicyIteration(env, pi, gamma, tol = 1e-10):
    num_states = env.nS
    num_actions = env.nA
    iterations = 0
    
    while True:
        # print(pi)
        iterations += 1
        pi_old = pi
        v = EvaluatePolicy(env, pi_old, gamma, tol)
        pi = ImprovePolicy(env, v, gamma)
        
        is_equal = True
        for s in range(num_states):
            if np.argmax(pi_old[s]) == np.argmax(pi[s]): 
                continue
            is_equal = False
        if is_equal == True:
            break
    return pi, v, iterations

 

### Testing Policy Iteration 

In [8]:
gamma = 0.9
env = CareerPathEnv()

print("Initial Policy: \n",policy_random)
pi, v, iters = PolicyIteration(env, policy_random, gamma)
print("Final Policy: \n",pi)
print("State Value Function: ",v)
print("Number of iterations for Policy Iteration: ",iters)

# average number of iterations required
avg_iters = 0
min_iters = 1000
max_iters = 0
for _ in range(100):
    policy_random = np.array((np.random.permutation(2), np.random.permutation(2), np.random.permutation(2), np.random.permutation(2)))
    _, _, iters = PolicyIteration(env,policy_random, gamma)
    avg_iters += iters
    min_iters = min(min_iters, iters)
    max_iters = max(max_iters, iters)
avg_iters /= 100
print("Iterations:")
print("Min\t", "Max\t" , "Average")
print(min_iters,"\t", max_iters,"\t", avg_iters)

Initial Policy: 
 [[0 1]
 [1 0]
 [0 1]
 [0 1]]
Final Policy: 
 [[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]
State Value Function:  [1000. 1000. 1000. 1000.]
Number of iterations for Policy Iteration:  2
Iterations:
Min	 Max	 Average
1 	 2 	 1.89


***

### A1. Find an optimal policy to navigate the given environment using Value Iteration (VI)

<img src='assets/value_iteration.png' width="500" align="left"></img>

In [9]:
# write your code here


### Testing Value Iterations

In [10]:
# write your code for testing value iteration here


### A1.2 Compare PI and VI in terms of convergence (average number of iteration, time required for each iteration). Is the policy obtained by both same?

Write your answer here

***

## Part B : A Stochastic Career Path

Now consider a more realistic Markov Decision Process below with four states and two actions available at each state. In this setting Actions have nondeterministic effects, i.e., taking an action in a state always leads to one next state, but which state is the one next state is determined by transition probabilities. These transition probabilites are shown in the figure attached to the transition arrows from states and actions to states. There are two actions out of each state for the agent to choose from: D for development and R for research. The same _ultimately-care-only-about-money_ reward scheme is given along with the states.

<img src='assets/mdp-nd.png' width="700" align="left"></img>

In [11]:
'''
Represents a Career Path problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticCareerPathEnv(gym.Env):
    '''
    StocasticCareerPathEnv represents the Gym Environment for the Career Path problem environment
    States : [0:'Unemployed',1:'Industry',2:'Grad School',3:'Academia']
    Actions : [0:'Research', 1:'Development']
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=3,no_states=4,no_actions=2):
        '''
        Constructor for the CareerPath class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 4
            no_actions : The no. of possible actions which is 2
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.prob_dynamics = {
            # s: {
            #   a: [(p(s,s'|a), s', r', terminal/not), (p(s,s''|a), s'', r'', terminal/not)]    
            # }
            
            0: {
                0: [(1.0, 2, 0.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            1: {
                0: [(0.9, 0, -10.0, False),(0.1, 1, 100, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            2: {
                0: [(0.9, 3, 10.0, False),(0.1, 2, 0, False)],
                1: [(0.9, 1, 100.0, False),(0.1, 1, 100, False)],
            },
            3: {
                0: [(1.0, 3, 10.0, False)],
                1: [(0.9, 1, 100.0, False),(0.1, 3, 10, False)],
            },
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(1.0, 2, 0.0, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            1: {
                0: [(0.9, 0, -10.0, False),(0.1, 1, 100, False)],
                1: [(1.0, 1, 100.0, False)],
            },
            2: {
                0: [(0.9, 3, 10.0, False),(0.1, 2, 0, False)],
                1: [(0.9, 1, 100.0, False),(0.1, 1, 100, False)],
            },
            3: {
                0: [(1.0, 3, 10.0, False)],
                1: [(0.9, 1, 100.0, False),(0.1, 3, 10, False)],
            },
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        if self.state == 0 or (self.state == 1 and action == 1) or (self.state == 3 and action == 0):
            index = 0    
        else:
            index = np.random.choice(2,1,p=[0.9,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

### Navigating in Stochastic Career Path 

In [12]:
env = StochasticCareerPathEnv()
is_Terminal = False
start_state = env.reset()
steps = 0
total_reward = 0

# you may change policy here
policy = policy_random
# policy = policy_1
# policy = policy_2

print("State\t", "Action\t" , "New State\t" , "Reward\t" , "is_Terminal")
steps = 0
max_steps = 5

prev_state = start_state

while steps < 10:
    steps += 1

    action = np.random.choice(2,1,p=policy[prev_state])[0]  #0 -> Research, 1 -> Development
    state, reward, is_Terminal = env.step(action)
    
    total_reward += reward

    print(" ",prev_state, "\t  ", action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    prev_state = state
    
print("Total Number of steps:", steps)
print("Final Reward:", total_reward)

State	 Action	 New State	 Reward	 is_Terminal
  3 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
  1 	   1 	   1 	 100.0 	   False
Total Number of steps: 10
Final Reward: 1000.0


### B1.1 Find an optimal policy to navigate the given SCP environment using Policy Iteration (PI) 

In [13]:
# [Hint] What would change for the stochastic MDP in the Policy Iteration code from Part A?
# write your code here

### B1.2 Find an optimal policy to navigate the given SCP environment using Value Iteration (VI) 

In [14]:
# [Hint] What would change for the stochastic MDP in the Value Iteration code from Part A?
# write your code here 

### B1.3  Compare PI and VI in terms of convergence (average number of iteration, time required for each iteration). Is the policy obtained by both same for SCP environment?


In [15]:
# write your code for comparison here

Write your comments compairing convergence and policies here.